<a href="https://colab.research.google.com/github/kiki-glitch/DSA-Tutorials-Python/blob/main/Dynamic_Arrays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import annotations
from typing import Generic, Iterator, List, Optional, TypeVar

In [2]:
T = TypeVar('T')

In [9]:
from __future__ import annotations
from typing import Generic, Iterator, List, Optional, TypeVar

T = TypeVar("T")

class DynamicArray(Generic[T]):
    """
    A generic dynamic array implementation in Python.
    - Starts with a given capacity (default 16)
    - Doubles capacity on growth when full
    - Shrinks only when removing via remove_at (to len-1 as per the Java code)
    """

    def __init__(self, capacity: int = 16) -> None:
        if capacity < 0:
            raise ValueError(f"Illegal Capacity: {capacity}")
        self._capacity: int = capacity
        self._len: int = 0
        # Internal storage is a fixed-size list padded with None
        self._arr: List[Optional[T]] = [None] * capacity

    # --- size & emptiness ---
    def size(self) -> int:
        return self._len

    def __len__(self) -> int:
        return self._len

    def is_empty(self) -> bool:
        return self._len == 0

    # --- index access ---
    def _check_bounds(self, index: int) -> None:
        if index < 0 or index >= self._len:
            raise IndexError("Index out of bounds")

    def get(self, index: int) -> T:
        self._check_bounds(index)
        # We stored Optional[T], but bounds guarantee non-None for assigned slots
        return self._arr[index]  # type: ignore[return-value]

    def __getitem__(self, index: int) -> T:
        return self.get(index)

    def set(self, index: int, elem: T) -> None:
        self._check_bounds(index)
        self._arr[index] = elem

    def __setitem__(self, index: int, elem: T) -> None:
        self.set(index, elem)

    # --- clear ---
    def clear(self) -> None:
        for i in range(self._len):
            self._arr[i] = None
        self._len = 0

    # --- add / append with automatic resize ---
    def add(self, elem: T) -> None:
        # Time to resize?
        if self._len + 1 >= self._capacity:
            if self._capacity == 0:
                self._capacity = 1
            else:
                self._capacity *= 2
            new_arr: List[Optional[T]] = [None] * self._capacity
            for i in range(self._len):
                new_arr[i] = self._arr[i]
            self._arr = new_arr

        self._arr[self._len] = elem
        self._len += 1

    def append(self, elem: T) -> None:
        """Pythonic alias for add()."""
        self.add(elem)

    # --- remove by index ---
    def remove_at(self, rm_index: int) -> T:
        self._check_bounds(rm_index)
        data = self._arr[rm_index]  # Optional[T], but truly assigned
        # Create a new array with size len-1 (matching the Java behavior)
        new_arr: List[Optional[T]] = [None] * (self._len - 1)
        j = 0
        for i in range(self._len):
            if i == rm_index:
                continue
            new_arr[j] = self._arr[i]
            j += 1
        self._arr = new_arr
        self._len -= 1
        self._capacity = self._len  # mirrors: capacity = --len
        return data  # type: ignore[return-value]

    # --- remove by value ---
    def remove(self, obj: object) -> bool:
        idx = self.index_of(obj)
        if idx == -1:
            return False
        self.remove_at(idx)
        return True

    # --- search ---
    def index_of(self, obj: object) -> int:
        for i in range(self._len):
            if obj is None:
                if self._arr[i] is None:
                    return i
            else:
                if obj == self._arr[i]:
                    return i
        return -1

    def contains(self, obj: object) -> bool:
        return self.index_of(obj) != -1

    # --- iterator ---
    def __iter__(self) -> Iterator[T]:
        index = 0
        while index < self._len:
            # _arr[index] is Optional[T], but within len it’s assigned
            yield self._arr[index]  # type: ignore[misc]
            index += 1

    # --- string / repr ---
    def __repr__(self) -> str:
        if self._len == 0:
            return "[]"
        # Build like Java's toString: [a, b, c]
        parts = []
        for i in range(self._len):
            parts.append(repr(self._arr[i]))
        return "[" + ", ".join(parts) + "]"

    def __str__(self) -> str:
        # Human-friendly without quotes for basic types
        if self._len == 0:
            return "[]"
        parts = []
        for i in range(self._len):
            parts.append(str(self._arr[i]))
        return "[" + ", ".join(parts) + "]"


In [11]:
da = DynamicArray() # Create an instance of DynamicArray
da.add(10)
da.add(20)
da.add(30)        # triggers resize
print(da.size())  # 3
print(da[1])      # 20
da[1] = 99
print(da)         # [10, 99, 30]
print(da.index_of(99))  # 1
da.remove(10)
print(da)         # [99, 30]
val = da.remove_at(0)
print(val, da)    # 99 [30]

3
20
[10, 99, 30]
1
[99, 30]
99 [30]
